### Can tidal forces and the Lidov-Kozai mechanism cause a system of exomoons to align in the same orbital plane?

In [1]:
# packages
import numpy as np
from amuse.lab import Particles
import matplotlib.pyplot as plt
from amuse.units.constants import G
from amuse.units import units, constants
from amuse.lab import Huayno, nbody_system
from amuse.community.ph4.interface import ph4
from amuse.ext.orbital_elements import get_orbital_elements_from_binary

In [2]:
# Creating a SIMPLE system of the Sun, Jupiter and Ganymede (largest moon) --> begin with Sun and Jupiter
system = Particles(2)

# Declaring Sun
sun = system[0]
sun.name = 'Sun'
sun.mass = 1 | units.MSun
sun.position = (0,0,0) | units.au
sun.velocity = (0,0,0) | units.kms

# Declaring Jupiter
jupiter = system[1]
jupiter.name = 'Jupiter'
jupiter.mass = 1 | units.MJupiter
jupiter.position = (5.2044,0,0) | units.au

# Relative velocities
def relative_orbital_velocity(mass, distance):
    return (G*mass/distance).sqrt()

# Jupiter velocity
vorb_j = relative_orbital_velocity(sun.mass+jupiter.mass, jupiter.position.sum())
jupiter.velocity = (0, 1, 0) * vorb_j

# Centering the system
system.move_to_center()

# Declaring Ganymede
ganymede = Particles(1)
ganymede.name = 'Ganymede'
ganymede.mass = 1.48E23 | units.kg

#initial parameters --> will depend on those needed for Lidov-Kozai Mechanism to take effect
ganymede.position = (1070400,0,0) | units.km 
ganymede.inclination = np.pi | units.rad
ganymede.eccentricity = 0.3

# Ganymede velocity
vorb_g = relative_orbital_velocity(ganymede.mass+jupiter.mass, ganymede.position.sum())
ganymede.velocity = (0, 1, 0) * vorb_g 

# We want Ganymede to orbit around Jupiter --> must add the positions and velocity of Jupiter to the Ganymede's
ganymede.position += jupiter.position
ganymede.velocity += jupiter.velocity

# Adding Ganymede and recentering the system
system.add_particle(ganymede)
system.move_to_center()

In [3]:
print(system)

                 key  eccentricity  inclination         mass         name           vx           vy           vz            x            y            z
                   -         none          rad         MSun         none          kms          kms          kms           au           au           au
====================  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========
 1069499711612974690    0.000e+00    0.000e+00    1.000e+00          Sun    0.000e+00   -1.246e-02    0.000e+00   -4.964e-03    0.000e+00    0.000e+00
10151226502096496151    0.000e+00    0.000e+00    9.546e-04      Jupiter    0.000e+00    1.305e+01    0.000e+00    5.199e+00    0.000e+00    0.000e+00
11095659992137875480    3.000e-01    3.142e+00    7.441e-08     Ganymede    0.000e+00    2.393e+01    0.000e+00    5.207e+00    0.000e+00    0.000e+00
====================  ===========  ===========  ===========  ===========  ===========  ======

In [4]:
help(get_orbital_elements_from_binary)

Help on function get_orbital_elements_from_binary in module amuse.ext.orbital_elements:

get_orbital_elements_from_binary(binary, G=quantity<1.0 length**3 / (mass * time**2)>)
    Function that computes orbital elements from given two-particle set.
    Elements are computed for the second particle in this set and the
    return values are: mass1, mass2, semimajor axis, eccentricity,
    cosine of true anomaly, cosine of inclination, cosine of the
    longitude of the ascending node and the cosine of the argument of
    pericenter. In case of a perfectly circular orbit the true anomaly
    and argument of pericenter cannot be determined; in this case the
    return values are 1.0 for both cosines.



In [5]:
# Integrate system over time
def integrate_system(particles, dt, end_time):
    convert_nbody = nbody_system.nbody_to_si(particles.mass.sum(), particles[1].position.length())
    
    gravity = Huayno(convert_nbody)
    gravity.particles.add_particles(particles)
    
    ecc = [] #eccentricity
    inc = [] #inclination
    time = []
    
    channel = gravity.particles.new_channel_to(particles)
    
    while gravity.model_time < end_time:
        
        channel.copy()
        jup_gan_binary = particles[1:3]
        jup_gan_oe = get_orbital_elements_from_binary(jup_gan_binary, G=constants.G)
        
        gravity.evolve_model(gravity.model_time + (dt))
        ecc.append(jup_gan_oe[3])
        inc.append(jup_gan_oe[5])
        time.append(gravity.model_time + (dt))
        
    gravity.stop()
    
    return ecc, inc, time

In [ ]:
ecc, inc, time = integrate_system(system, 10|units.day, 0.1|units.Myr)

In [19]:
# eccentricity as function of the orbital inclination
def plot_track(ecc, inc, time):
    
    test = inc/np.radians
    
    fig, ax2 = plt.subplots(figsize=[10, 10])
    
    #ax1.minorticks_on()
    #ax1.locator_params(nbins=3)
    #ax1.scatter(time.value_in(units.yr), ecc.value_in(units), color = 'g')
    
    ax2.minorticks_on()
    ax2.locator_params(nbins=3)
    ax2.scatter(time.value_in(units.yr), test.value_in(units.rad), color = 'g')
   
    plt.show()

In [7]:
#plot_track(ecc, inc, time)